# Bitcoin Forecasting Prompt Generation (Ollama Example)

This notebook demonstrates how to transform historical BTC/USDT price data into prompt–completion pairs that can be used for **fine-tuning or prompting a large language model (LLM)**.

It aligns with the project goal of using Ollama to **forecast short-term price movements** based on past trends.

We will:

1. Load historical BTC closing prices
2. Construct prompts from the past 60 minutes
3. Attach the next closing price as the target (completion)
4. Format and write to a `.jsonl` file for training or testing

We do **not** actually run the LLM in this notebook — instead, we prepare high-quality inputs **for fine-tuning** or **prompt-response evaluation**.

In [1]:
import json
from pathlib import Path
import pandas as pd

## Step 1 – Load Historical Data

We use a historical BTC/USDT CSV file exported from Binance or another exchange.

Assumptions:
- The CSV file is located at `historical_analysis/data/historical_btc_data.csv`
- It has a `start` timestamp column (in milliseconds)
- It has a `close` column for the closing price

In [ ]:
# Constants
INPUT_CSV = Path("data/historical_btc_data.csv")
WINDOW = 60  # Use last 60 closes as context

# Load CSV
df = pd.read_csv(INPUT_CSV)

# Convert timestamp and clean
df["Time"] = pd.to_datetime(df["start"], unit="ms", utc=True)
series = df.set_index("Time")["close"].astype(float).dropna()

# Preview
series.head()

## Step 2 – Create Prompts from Price Windows

We'll create a helper function that:
- Takes a list of 60 prices
- Formats them as a numbered list
- Builds a **prompt** string asking the model to forecast the next price

In [ ]:
def make_example(prices):
    """
    Given a list of floats, builds a prompt string listing them and a completion
    asking for the next price.
    """
    lines = [f"{i+1:02d}. {p:.2f}" for i, p in enumerate(prices)]
    prompt = (
        "Here are the last 60 BTC/USDT closing prices (most recent last):\n"
        + "\n".join(lines)
        + "\n\nPlease forecast the next closing price (just the number)."
    )
    return prompt

## Step 3 – Generate Prompt–Completion Pairs

For every sequence of 60 prices, we attach the **next closing price** as the label.  
This structure is used for both **prompt-based forecasting** and **fine-tuning**.

We will generate a few samples below and inspect them.

In [ ]:
samples = []

for i in range(3):  # Just create 3 examples for preview
    hist = series.iloc[i : i + WINDOW].tolist()
    target = series.iloc[i + WINDOW]
    prompt = make_example(hist)
    completion = f" {target:.2f}"  # space prefix is required
    samples.append({
        "prompt": prompt,
        "completion": completion
    })

samples[0]  # Show first example


## Sample Output

This is a single prompt–completion pair that we would send to an LLM or use to fine-tune a forecasting head.

- The **prompt** gives the last 60 BTC closing prices in order.
- The **completion** is a single number: the next price.

This lets the model learn short-term patterns from raw prices.


In [ ]:
# Pretty print all three samples
for i, sample in enumerate(samples, 1):
    print(f"\n--- Sample #{i} ---")
    print("Prompt:\n", sample["prompt"])
    print("Completion:\n", sample["completion"])

## Step 4 – Write to JSONL (Optional)

This step saves all prompt–completion pairs into a `.jsonl` file named `finetune_data.jsonl`.

This format is compatible with:
- Ollama fine-tuning
- OpenAI fine-tune APIs
- Any system that takes "prompt" → "completion" training pairs

In [ ]:
OUTPUT_JSONL = Path("finetune_data.jsonl")

with open(OUTPUT_JSONL, "w") as out:
    for i in range(len(series) - WINDOW):
        hist = series.iloc[i : i + WINDOW].tolist()
        target = series.iloc[i + WINDOW]
        prompt = make_example(hist)
        completion = f" {target:.2f}"
        out.write(json.dumps({
            "prompt": prompt,
            "completion": completion
        }) + "\n")

print(f"Wrote {OUTPUT_JSONL}")

## Summary

- This notebook demonstrates how to prepare time-series data for LLM training or inference.
- The format follows Ollama and OpenAI fine-tune requirements.
- The `.jsonl` output can be used to:
  - Fine-tune a local model (e.g. Mistral, LLaMA)
  - Evaluate prompt performance on holdout data
  - Generate forecasts based on prompt-only workflows

> You can now move to a model script or dashboard and test how the LLM performs on these prompts!